In [1]:
# Import necessary libraries
import os
import numpy as np
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from google.colab import drive
drive.mount('/content/drive')

# Data directories
train_dir = '/content/drive/MyDrive/DR_50/Train'
test_dir = '/content/drive/MyDrive/DR_50/Test'
valid_dir = '/content/drive/MyDrive/DR_50/Valid'
# Image dimensions
img_width, img_height = 300, 300  # Adjust according to your image dimensions
batch_size = 32
num_epochs = 20
# Load model for feature extraction
model = ResNet50(weights='imagenet', include_top=False)
# Function to extract features from an image
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = model.predict(img_data)
    return features
# Load images and extract features
x_train, y_train = [], []
for i in range(5):
    class_dir = os.path.join(train_dir, str(i))
    for img_path in os.listdir(class_dir):
        features = extract_features(os.path.join(class_dir, img_path))
        x_train.append(features)
        y_train.append(i)
# Evaluate on test set
x_test, y_test = [], []
for i in range(5):
    class_dir = os.path.join(test_dir, str(i))
    for img_path in os.listdir(class_dir):
        features = extract_features(os.path.join(class_dir, img_path))
        x_test.append(features)
        y_test.append(i)

# Flatten the features
x_train_flat = np.array([feature.flatten() for feature in x_train])
x_test_flat = np.array([feature.flatten() for feature in x_test])

# Train and evaluate Naive Bayes
print("Training and evaluating Naive Bayes...")
# Train Naive Bayes classifier
nb_classifier = GaussianNB()
nb_classifier.fit(x_train_flat, y_train)

# Predict on test set
y_pred_nb = nb_classifier.predict(x_test_flat)

# Print performance metrics for Naive Bayes
print('Performance metrics for Naive Bayes:')
print('Classification Report:')
print(metrics.classification_report(y_test, y_pred_nb))

# Additional performance metrics
accuracy_nb = metrics.accuracy_score(y_test, y_pred_nb)
sensitivity_nb = metrics.recall_score(y_test, y_pred_nb, average='macro')
precision_nb = metrics.precision_score(y_test, y_pred_nb, average='macro')
f1_nb = metrics.f1_score(y_test, y_pred_nb, average='macro')

print(f'Accuracy: {accuracy_nb}')
print(f'Sensitivity: {sensitivity_nb}')
print(f'Precision: {precision_nb}')
print(f'F1 Score: {f1_nb}\n')


Mounted at /content/drive
1/1 [==============================] - 0s 219ms/step
Training and evaluating Naive Bayes...
Performance metrics for Naive Bayes:
Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.40      0.44         5
           1       1.00      0.60      0.75         5
           2       0.60      0.60      0.60         5
           3       0.20      0.20      0.20         5
           4       0.62      1.00      0.77         5

    accuracy                           0.56        25
   macro avg       0.59      0.56      0.55        25
weighted avg       0.58      0.56      0.55        25

Accuracy: 0.56
Sensitivity: 0.5599999999999999
Precision: 0.5850000000000001
F1 Score: 0.5527350427350427

